# **part1**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
json_file_path = '/content/drive/My Drive/sarcasm.json'

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer


import nltk
nltk.download('punkt')
nltk.download('stopwords')


df = pd.read_json('/content/drive/My Drive/sarcasm.json', lines=True)

# function for cleaning data
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

# apply cleaning function to each headline
df['cleaned_headline'] = df['headline'].apply(clean_text)

# removing stopwords from cleaned headlines
stop_words = set(stopwords.words('english'))
df['cleaned_headline'] = df['cleaned_headline'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


# extracting labels to find if the headline is sarcastic
y = df['is_sarcastic'].values

# split the dataset into training and test sets
train_df, eval_df, y_train, y_eval = train_test_split(df, y, test_size=0.2, random_state=42)

# feature extraction with TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # maximum of 1000 features
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['cleaned_headline']).toarray()  # applying TF-IDF to training data


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **part2**

In [5]:
!unzip "/content/drive/My Drive/glove.6B.zip" -d "/content/drive/My Drive/glove.6B"


Archive:  /content/drive/My Drive/glove.6B.zip
replace /content/drive/My Drive/glove.6B/glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score


In [8]:
tokenizer = Tokenizer(num_words=1000)  # limit vocab to top 1000 words

# learning vocab from cleaned headlines in train_df
tokenizer.fit_on_texts(train_df['cleaned_headline'])

# converting the cleaned headlines into sequences of integers
X_train_seq = tokenizer.texts_to_sequences(train_df['cleaned_headline'])  # For training data
X_eval_seq = tokenizer.texts_to_sequences(eval_df['cleaned_headline'])  # For evaluation data

# find the maximum sequence length
max_length = max(max(len(seq) for seq in X_train_seq), max(len(seq) for seq in X_eval_seq))

# pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')  # training data
X_eval_pad = pad_sequences(X_eval_seq, maxlen=max_length, padding='post')  # evaluation data

# a mapping of each word to its unique index
word_index = tokenizer.word_index


# **part3**

In [9]:
def load_glove_embeddings(file_path):
    # a dictionary to store the word embeddings.
    embedding_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # split each line into word and its corresponding vector
            values = line.split()
            word = values[0]  # word is first value
            # rest of values form the vector, converted to floats
            coefs = np.asarray(values[1:], dtype='float32')
            # store the word and its vector
            embedding_index[word] = coefs
    return embedding_index
# dimensions of GloVe vectors
glove_dimensions = [50, 100, 200, 300]
glove_paths = {
    50: '/content/drive/My Drive/glove.6B/glove.6B.50d.txt',
    100: '/content/drive/My Drive/glove.6B/glove.6B.100d.txt',
    200: '/content/drive/My Drive/glove.6B/glove.6B.200d.txt',
    300: '/content/drive/My Drive/glove.6B/glove.6B.300d.txt'
}
# func to create embedding matrix
def create_embedding_matrix(dim, word_index):
    # load GloVe embeddings for specified dimension.
    embedding_index = load_glove_embeddings(glove_paths[dim])
    embedding_matrix = np.zeros((len(word_index) + 1, dim))
    for word, i in word_index.items():
        # corresponding GloVe vector
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


In [10]:
# func to calculate average embedding for each sequence
def get_average_embedding(sequences, embedding_matrix):

    embeddings = np.zeros((len(sequences), embedding_matrix.shape[1]))
    for i, seq in enumerate(sequences):
        # filter out zeros to find actual word indices
        non_zero_elements = [idx for idx in seq if idx != 0]
        if non_zero_elements:
            # retrieve and average the embeddings for the non zero elements
            seq_embeddings = np.array([embedding_matrix[idx] for idx in non_zero_elements])
            embeddings[i] = np.mean(seq_embeddings, axis=0)
    return embeddings




In [11]:

for dim in glove_dimensions:
    print(f"Evaluating model with GloVe {dim}d embeddings...")
    # creating  embedding matrix for current dim
    embedding_matrix = create_embedding_matrix(dim, word_index)
    # converting training and evaluation sequences to their average embeddings
    X_train_avg = get_average_embedding(X_train_pad, embedding_matrix)
    X_eval_avg = get_average_embedding(X_eval_pad, embedding_matrix)

    # initialize and train Logistic Regression model.
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train_avg, y_train)

    # predict and evaluate the model
    y_pred = model.predict(X_eval_avg)


    print(f"Results for GloVe {dim}d embeddings:")
    print(f"F1 Score: {f1_score(y_eval, y_pred)}")
    print(f"Precision: {precision_score(y_eval, y_pred)}")
    print(f"Recall: {recall_score(y_eval, y_pred)}\n")


Evaluating model with GloVe 50d embeddings...
Results for GloVe 50d embeddings:
F1 Score: 0.6027554535017221
Precision: 0.6307569082899479
Recall: 0.5771344814950531

Evaluating model with GloVe 100d embeddings...
Results for GloVe 100d embeddings:
F1 Score: 0.6162347560975611
Precision: 0.6419213973799127
Recall: 0.5925247343349213

Evaluating model with GloVe 200d embeddings...
Results for GloVe 200d embeddings:
F1 Score: 0.6387818041634541
Precision: 0.6738511590077267
Recall: 0.6071821179919384

Evaluating model with GloVe 300d embeddings...
Results for GloVe 300d embeddings:
F1 Score: 0.6594427244582043
Precision: 0.6986469864698647
Recall: 0.6244045437889337

